In [4]:
## import all dependencies

import numpy as np
import pandas as pd
import _pickle as cPickle
from collections import defaultdict
import re
import sys
import os

os.environ['KERAS_BACKEND']='theano'

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, merge, Dropout
from keras.models import Model

In [6]:
## param list

## parameter list

MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 200
DROPOUT_RATE    = 0.4
HIDDEN_UNITS    = 200

# NB_CLASSES      = 2

# LEARNING
# BATCH_SIZE      = 100
# NB_EPOCHS       = 10
# RUNS            = 5
# VAL_SIZE        = 0.2

VALIDATION_SPLIT = 0.2

In [7]:
## data preprocessing
import pandas as pd
from nltk.corpus import stopwords
import string,re

dataset=pd.read_csv('train.tsv',delimiter='\t',encoding='utf-8')
dataset.columns=['ID','Label','Statement','Subject','speaker','job_title',
           'state_info','pantry_affiliation','barely_true_cnt','false_cnt',
           'half_true_cnt','mostly_true_cnt','pants_on_fire_cnt','Context']

def preprocessing_txt(dataset):
    stop_words = set(stopwords.words('english'))
    rm_punct = re.compile('[{}]'.format(re.escape(string.punctuation)))
    corpus=[]
    for elm in range(0, len(dataset.index)):
        res=' '.join([i for i in dataset['Statement'][elm].split() if i not in stop_words])
        res=rm_punct.sub(' ', res)
        corpus.append(res)
    return corpus

corpus=preprocessing_txt(dataset)

In [72]:
## import local dependencies
import util
from nlp_util import NLP_Task
import preprocessing
import importlib
importlib.reload(preprocessing)


## label
multi_labels = {'false':0, 'true':1,'pants-fire':2,'barely-true':3,'half-true':4,'mostly-true':5}
binary_labels = {'false':1, 'true':-1,'pants-fire':1,'barely-true':1,'half-true':0,'mostly-true':-1}

dataset['b_label'] = np.array(preprocessing.create_labels(labels=dataset['Label'].values,label_values=binary_labels))
dataset['m_label'] = np.array(preprocessing.create_labels(labels=dataset['Label'].values,label_values=multi_labels))

texts = corpus
labels = dataset['b_label']

labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', dataset.shape)
print('Shape of label tensor:', labels.shape)

#
# indices = np.arange(dataset.shape[0])
# np.random.shuffle(indices)


Shape of data tensor: (10239, 16)
Shape of label tensor: (10239, 2)


In [73]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

# cv = CountVectorizer()
# corpus = cv.fit_transform(corpus).toarray()

X_train, X_val, y_train, y_val  = train_test_split(corpus,
                                                   labels,train_size=0.8, random_state=123)


print('Number of positive and negative reviews in traing and validation set ')
print(y_train.sum(axis=0))
print(y_val.sum(axis=0))

c:\users\jvrat\appdata\local\programs\python\python36\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Number of positive and negative reviews in traing and validation set 
[1673. 6518.]
[ 441. 1607.]


In [ ]:
# GLOVE_DIR = "/ext/home/analyst/Testground/data/glove"
# embeddings_index = {}
# f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'))
# for line in f:
#     values = line.split()
#     word = values[0]
#     coefs = np.asarray(values[1:], dtype='float32')
#     embeddings_index[word] = coefs
# f.close()

# print('Total %s word vectors in Glove 6B 100d.' % len(embeddings_index))

# embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
# for word, i in word_index.items():
#     embedding_vector = embeddings_index.get(word)
#     if embedding_vector is not None:
#         # words not found in embedding index will be all-zeros.
#         embedding_matrix[i] = embedding_vector
        
# embedding_layer = Embedding(len(word_index) + 1,
#                             EMBEDDING_DIM,
#                             weights=[embedding_matrix],
#                             input_length=MAX_SEQUENCE_LENGTH,
#                             trainable=True)

In [2]:
## build glove-vec embedding layer
def build_glove_embedding_layers():
    embed_index={}
    with open('glove.6B.300d.txt'.format(EMBEDDING_DIM), encoding="utf8",) as file:
        for line in file:
            values=line.split()
            word=values[0]
            coefs=np.asarray(values[1:], dtype='float32')
            embed_index[word]=coefs
    embed_matrix=np.zeros((MAX_NB_WORDS, EMBEDDING_DIM))
    for word, indx in tokenized.word_index.items():
        if indx >= MAX_NB_WORDS:
            continue
        embed_vec=embed_index.get(word)
        if embed_vec is not None:
            embed_matrix[indx]=embed_vec
    return Embedding(input_dim=MAX_NB_WORDS,
                     output_dim=EMBEDDING_DIM,
                     input_length=MAX_SEQUENCE_LENGTH,
                     weights=[embed_matrix],
                     trainable=True, name='word embedding')

##
embedding_layers=build_glove_embedding_layers()

NameError: name 'EMBEDDING_DIM' is not defined

In [1]:
embedding_layes

NameError: name 'embedding_layes' is not defined

In [ ]:
# applying complex convolutional approach
convs = []
filter_sizes = [3,4,5]

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

for fsz in filter_sizes:
    l_conv = Conv1D(nb_filter=128,filter_length=fsz,activation='relu')(embedded_sequences)
    l_pool = MaxPooling1D(5)(l_conv)
    convs.append(l_pool)
    
l_merge = Merge(mode='concat', concat_axis=1)(convs)
l_cov1= Conv1D(128, 5, activation='relu')(l_merge)
l_pool1 = MaxPooling1D(5)(l_cov1)
l_cov2 = Conv1D(128, 5, activation='relu')(l_pool1)
l_pool2 = MaxPooling1D(30)(l_cov2)
l_flat = Flatten()(l_pool2)
l_flat = Dense(HIDDEN_UNITS)(l_flat)
l_flat = Dropout(DROPOUT_RATE)(l_flat)
l_dense = Dense(128, activation='relu')(l_flat)

preds = Dense(2, activation='softmax')(l_dense)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

print("model fitting - more complex convolutional neural network")
model.summary()
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          nb_epoch=20, batch_size=50)